In [2]:
import tools
import datetime
import numpy as np

In [113]:
df = tools.load_dataframe()

In [114]:
df.head()

,DAY,HOUR,OPCAT_CATEGORY,SIZE,SUBBRANCH_ID
0,2014-10-25,16,SM,26,1005609
1,2014-04-14,11,SM,21,1005625
2,2014-10-25,11,SM,36,1005609
3,2015-08-14,12,CM,175,1006034
4,2014-04-14,12,SM,29,1005625


# Useful functions

In [15]:
def select(df, SUBBRANCH_ID, OPCAT_CATEGORY):
    return df[(df["SUBBRANCH_ID"] == SUBBRANCH_ID) & (
        (df["OPCAT_CATEGORY"] == OPCAT_CATEGORY) | (df["OPCAT_CATEGORY"] == "?"))][["DAY", "HOUR", "SIZE"]]

In [17]:
select(df, 1005609, "CM").head()

,DAY,HOUR,SIZE
27,2015-11-09,19,69
31,2015-11-09,18,139
95,2015-02-10,9,90
96,2015-02-10,8,21
101,2015-03-18,9,95


# Baseline: predict average value

In [129]:
dates = [
    datetime.datetime.strptime("01.03.2015", "%d.%m.%Y").date(),
    datetime.datetime.strptime("01.04.2015", "%d.%m.%Y").date(),
    datetime.datetime.strptime("01.08.2015", "%d.%m.%Y").date(),
    datetime.datetime.strptime("01.09.2015", "%d.%m.%Y").date(),
    datetime.datetime.strptime("01.10.2015", "%d.%m.%Y").date(),
]
def filter_standart_work_hours(df, start_time=8, end_time=19):
    return df[(df["HOUR"] >= start_time) & ((df["HOUR"] <= end_time))]

def filter_category(df, category):
    return df[(df["OPCAT_CATEGORY"] == category)]

def make_train_test(df, bounder_data, acc, delta=datetime.timedelta(days=31), short_history=None):    
    train = df[df["DAY"] < bounder_data]
    test = df[(df["DAY"] >= bounder_data) & (df["DAY"] < bounder_data + delta)]
    
    if short_history:
        train = train[train["DAY"] > bounder_data - short_history*delta]

    train = filter_standart_work_hours(train)
    test = filter_standart_work_hours(test)
    
    for category in ["SM", "CM"]:
        acc_ = 0.
        number_elements = 0.
        for office in df["SUBBRANCH_ID"].value_counts().keys():

            train_ = select(train, office, category)
            test_ = select(test, office, category)
            
            train_model = train_.groupby(["HOUR"])["SIZE"].mean()
            for hour in xrange(8, 20):
                acc_ += np.sum((test_[test_["HOUR"] == hour]["SIZE"] - train_model[hour])**2)
                number_elements += len(test_[test_["HOUR"] == hour]["SIZE"])
                
        acc[category].append((acc_ / number_elements)**0.5)
    return acc

acc = {
    'SM' : [],
    'CM' : [],
}
for data in dates:
    make_train_test(df, data, acc)
print acc
print 'SM', np.array(acc['SM']).mean(), np.array(acc['SM']).std()
print 'CM', np.array(acc['CM']).mean(), np.array(acc['CM']).std()

{'CM': [34.541660476791748, 32.057648384617941, 30.622546210786037, 29.224790412562733, 30.0782920105868], 'SM': [9.578927614803181, 9.8780970753063162, 10.00994915379826, 8.9381003007681556, 9.0786161835398111]}
SM 9.49673806564 0.424847619244
CM 31.3049874991 1.86227163569


# Baseline: predict average without new year holidays

In [130]:
newyear_2014 = [
    datetime.datetime.strptime("15.12.2014", "%d.%m.%Y").date(),
    datetime.datetime.strptime("15.01.2015", "%d.%m.%Y").date()
]
newyear_2015 =  [
    datetime.datetime.strptime("15.12.2015", "%d.%m.%Y").date(),
    datetime.datetime.strptime("12.01.2016", "%d.%m.%Y").date()
]

In [131]:
def remove_interval(df, holidays):
    return df[(df["DAY"] < holidays[0]) | (df["DAY"] > holidays[1])]

df_ordinal = remove_interval(df, newyear_2014)
df_ordinal = remove_interval(df_ordinal, newyear_2015)

acc = {
    'SM' : [],
    'CM' : [],
}
for data in dates:
    make_train_test(df_ordinal, data, acc)
print acc
print 'SM', np.array(acc['SM']).mean(), np.array(acc['SM']).std()
print 'CM', np.array(acc['CM']).mean(), np.array(acc['CM']).std()

{'CM': [34.914407085942273, 32.30064541506303, 30.291982727736535, 29.17498237415613, 30.069831513066497], 'SM': [9.5190341785288233, 9.810352150248308, 9.9573458308977703, 8.895275544309218, 9.055403606801157]}
SM 9.44748226216 0.41362132981
CM 31.3503698232 2.05414982276


# Baseline: predict average without new year holidays (N month only)

In [132]:
df_ordinal = remove_interval(df, newyear_2014)
df_ordinal = remove_interval(df_ordinal, newyear_2015)

acc = {
    'SM' : [],
    'CM' : [],
}
for data in dates:
    make_train_test(df_ordinal, data, acc, datetime.timedelta(days=31), 1)
print acc
print 'SM', np.array(acc['SM']).mean(), np.array(acc['SM']).std()
print 'CM', np.array(acc['CM']).mean(), np.array(acc['CM']).std()

{'CM': [30.873986983289122, 31.717454503785831, 27.986701212180559, 29.259745925022475, 28.643288862463681], 'SM': [8.7566748132077539, 8.9714892400323212, 9.3788223819991856, 8.6436746285265222, 8.8182385089949058]}
SM 8.91377991455 0.255484598894
CM 29.6962354973 1.39241890747


In [133]:
for data in dates:
    make_train_test(df_ordinal, data, acc, datetime.timedelta(days=31), 3)
print acc
print 'SM', np.array(acc['SM']).mean(), np.array(acc['SM']).std()
print 'CM', np.array(acc['CM']).mean(), np.array(acc['CM']).std()

{'CM': [30.873986983289122, 31.717454503785831, 27.986701212180559, 29.259745925022475, 28.643288862463681, 30.665740968094859, 31.669740936102453, 28.482046818136499, 28.279395396859687, 29.249509767368938], 'SM': [8.7566748132077539, 8.9714892400323212, 9.3788223819991856, 8.6436746285265222, 8.8182385089949058, 8.8851925019017468, 8.7994309284482224, 9.1256402207714, 8.6451080787405186, 8.8389922820730877]}
SM 8.88632635847 0.213396436102
CM 29.6827611373 1.34937967071


In [134]:
for data in dates:
    make_train_test(df_ordinal, data, acc, datetime.timedelta(days=31), 6)
print acc
print 'SM', np.array(acc['SM']).mean(), np.array(acc['SM']).std()
print 'CM', np.array(acc['CM']).mean(), np.array(acc['CM']).std()

{'CM': [30.873986983289122, 31.717454503785831, 27.986701212180559, 29.259745925022475, 28.643288862463681, 30.665740968094859, 31.669740936102453, 28.482046818136499, 28.279395396859687, 29.249509767368938, 31.670404396544029, 31.955603410499339, 30.244898858479733, 28.19947856382182, 28.98247479158475], 'SM': [8.7566748132077539, 8.9714892400323212, 9.3788223819991856, 8.6436746285265222, 8.8182385089949058, 8.8851925019017468, 8.7994309284482224, 9.1256402207714, 8.6451080787405186, 8.8389922820730877, 9.3479278843922557, 9.1138419660382102, 8.9949315240992345, 8.5581542210582597, 8.8261271212206331]}
SM 8.9136164201 0.235536901541
CM 29.8586980929 1.41107553478


# Average between 1 month ago, 2 month ago, 3 month ago at this day

In [128]:
def average_previous_month_this_day(df, bounder_data, acc, delta=datetime.timedelta(days=28)):    
    train = df[df["DAY"] < bounder_data]
    test = df[(df["DAY"] >= bounder_data) & (df["DAY"] < bounder_data + delta)]
    
    for category in ["SM", "CM"]:
        acc_ = 0.
        number_elements = 0.
        for office in df["SUBBRANCH_ID"].value_counts().keys():

            train_ = select(train, office, category)
            test_ = select(test, office, category)
            
            train_model = train_.groupby(["HOUR"])["SIZE"].mean()
            
            for index, row in test_.iterrows():
                hour = row["HOUR"]
                model = 0
                elements = 0
                for k in [1, 2, 3]:
                    train_day = train_[train_["DAY"] == (row["DAY"] - datetime.timedelta(days=k*28))]
                    train_day_hour = train_day[train_day["HOUR"] == hour]
                    if (len(train_day_hour) > 0):
                        model += train_day_hour.iloc[0]["SIZE"]
                    if (len(train_day) == 0):
                        continue
                    elements += 1.
                model /= elements
                acc_ += (row["SIZE"] - model)**2
                number_elements += 1
                 
        acc[category].append((acc_ / number_elements)**0.5)
    return acc

acc = {
    'SM' : [],
    'CM' : [],
}
for data in dates:
    average_previous_month_this_day(df, data, acc)
print 'SM', np.array(acc['SM']).mean(), np.array(acc['SM']).std()
print 'CM', np.array(acc['CM']).mean(), np.array(acc['CM']).std()

SM 9.91909825757 0.317954632331
CM 27.8517293314 1.10314795562


# 12 weeks average at this week day

In [135]:
def average_previous_month_this_day(df, bounder_data, acc, delta=datetime.timedelta(days=28)):    
    train = df[df["DAY"] < bounder_data]
    test = df[(df["DAY"] >= bounder_data) & (df["DAY"] < bounder_data + delta)]
    
    if short_history:
        train = train[train["DAY"] > bounder_data - short_history*delta]

    for category in ["SM", "CM"]:
        acc_ = 0.
        number_elements = 0.
        for office in df["SUBBRANCH_ID"].value_counts().keys():

            train_ = select(train, office, category)
            test_ = select(test, office, category)
            
            train_model = train_.groupby(["HOUR"])["SIZE"].mean()
            
            for index, row in test_.iterrows():
                hour = row["HOUR"]
                model = 0
                elements = 0
                for k in range(1, 13):
                    train_day = train_[train_["DAY"] == (row["DAY"] - datetime.timedelta(days=k*7))]
                    train_day_hour = train_day[train_day["HOUR"] == hour]
                    if (len(train_day_hour) > 0):
                        model += train_day_hour.iloc[0]["SIZE"]
                    if (len(train_day) == 0):
                        continue
                    elements += 1.
                model /= elements
                acc_ += (row["SIZE"] - model)**2
                number_elements += 1
                 
        acc[category].append((acc_ / number_elements)**0.5)
    return acc

acc = {
    'SM' : [],
    'CM' : [],
}
for data in dates:
    average_previous_month_this_day(df, data, acc)
print 'SM', np.array(acc['SM']).mean(), np.array(acc['SM']).std()
print 'CM', np.array(acc['CM']).mean(), np.array(acc['CM']).std()

SM 8.8897681614 0.189153101741
CM 25.4834160897 1.42254802391


# Linear solution

In [153]:
from sklearn import linear_model

def use_linear_model(df, bounder_data, acc, delta=datetime.timedelta(days=28)):    
    train = df[df["DAY"] < bounder_data]
    test = df[(df["DAY"] >= bounder_data) & (df["DAY"] < bounder_data + delta)]
    
    for category in ["SM", "CM"]:
        acc_ = 0.
        number_elements = 0.
        for office in df["SUBBRANCH_ID"].value_counts().keys():

            train_ = select(train, office, category)
            test_ = select(test, office, category)
            
            train_model = train_.groupby(["HOUR"])["SIZE"].mean()
            
            for index, row in test_.iterrows():
                hour = row["HOUR"]
                X = []
                Y = []
                for k in range(1, 13):
                    train_day = train_[train_["DAY"] == (row["DAY"] - datetime.timedelta(days=k*7))]
                    train_day_hour = train_day[train_day["HOUR"] == hour]
                    if (len(train_day_hour) > 0):
                        X.append([k])
                        Y.append(train_day_hour.iloc[0]["SIZE"])
                    elif (len(train_day) > 0):
                        X.append([k])
                        Y.append(0)
                    
                    
                X = np.array(X).reshape((-1, 1))
                Y = np.array(Y)
                reg = linear_model.LinearRegression()
                reg.fit(X, Y)
                prediction = reg.predict([[0]])[0]
                
                acc_ += (row["SIZE"] - prediction)**2
                number_elements += 1
                 
        acc[category].append((acc_ / number_elements)**0.5)
    return acc

acc = {
    'SM' : [],
    'CM' : [],
}
for data in dates:
    use_linear_model(df, data, acc)
print 'SM', np.array(acc['SM']).mean(), np.array(acc['SM']).std()
print 'CM', np.array(acc['CM']).mean(), np.array(acc['CM']).std()

SM 11.3859353579 0.714527456027
CM 33.1885415762 2.7987646044
